<a href="https://colab.research.google.com/github/NickBourdeau/DS2020/blob/master/Modeling_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving hotel-booking-demand.zip to hotel-booking-demand.zip


In [4]:
from numpy import genfromtxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid') 
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
data = pd.read_csv('hotel-booking-demand.zip')
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


To see which months were the most popular, and to see the cancellation within these months, we will plot a histogram.


In [0]:
import seaborn as sns

import plotly.express as px
fig = px.histogram(data, x="arrival_date_month", color="is_canceled")
fig.show()



Decision Tree Model

In [0]:
training = data.drop(['agent', 'company', 'country', 'children', 'reservation_status'], axis = 1)
training_x = training.drop(['is_canceled'], axis = 1)
X = pd.get_dummies(training_x)
y = training['is_canceled']


In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

NameError: ignored

In [0]:
dtree = DecisionTreeClassifier(random_state=2, min_samples_leaf=5, min_samples_split=20)

dtree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=20,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=2, splitter='best')

In [0]:
print(dtree.score(X_train, y_train))
print(dtree.score(X_test, y_test))
print(accuracy_score(dtree.predict(X_test), y_test)) 

0.9078277540460303
0.841276488818159
0.841276488818159


Decision Tree Feature Importances

In [0]:
dtree.feature_importances_

array([0.10739929, 0.01481345, 0.03180332, ..., 0.        , 0.        ,
       0.        ])

In [0]:
importance = pd.DataFrame({'Importance': dtree.feature_importances_}, index=X_train.columns).sort_values('Importance', ascending = False)
importance

,Importance
deposit_type_Non Refund,0.315359
lead_time,0.107399
total_of_special_requests,0.068413
adr,0.060551
market_segment_Online TA,0.052924
...,...
reservation_status_date_2016-01-12,0.000000
reservation_status_date_2016-01-13,0.000000
reservation_status_date_2016-01-17,0.000000
reservation_status_date_2016-01-18,0.000000


In [0]:
importance.head()

,Importance
deposit_type_Non Refund,0.315359
lead_time,0.107399
total_of_special_requests,0.068413
adr,0.060551
market_segment_Online TA,0.052924


Visualizing the Decision Tree Importances

In [0]:
important = importance.head(20)


fig1 = px.bar(important, x=important.index, y = 'Importance')
fig1.show()



New Random Forest Model

In [0]:
train = data.drop(['agent', 'company', 'country', 'children', 'reservation_status'], axis = 1)
train_x = training.drop(['is_canceled'], axis = 1)
z = train_x[['deposit_type', 'lead_time', 'total_of_special_requests', 'adr', 'market_segment']]
Z = pd.get_dummies(z)
y = training['is_canceled']

Z_train, Z_test, y_train, y_test = train_test_split(Z, y, test_size=0.1, random_state=2)

forest1 = RandomForestClassifier(random_state=2)
forest1.fit(Z_train, y_train)
forest1_predictions = forest1.predict(Z_test)

In [0]:
forest1.score(Z_test, y_test)

0.8214255800318284

In [0]:
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, LeaveOneOut
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV

loocv_model = LogisticRegression(solver='saga')
loocv_scores = cross_val_score(loocv_model, Z, y, cv = 10)
loocv_scores.mean()

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.6/dist-packages/sklearn

0.6690677611190218

Random Forest Model

In [0]:
forest = RandomForestClassifier(random_state=2)

forest.fit(X_train, y_train)

forest_predictions = forest.predict(X_test)

In [0]:
forest.score(X_test, y_test)

0.9051846888349108

In [0]:
print('Decision Tree Score', dtree.score(X_test, y_test))
print('Random Forest Score', forest.score(X_test, y_test))

Decision Tree Score 0.841276488818159
Random Forest Score 0.9051846888349108


In [0]:
trained = training.drop(['reservation_status_date'], 1)
training1 = trained.iloc[:-12000]

dummies = training1.drop('is_canceled', 1)
train_X = pd.get_dummies(dummies)
train_y = training1['is_canceled']

training2 = trained.iloc[-12000:]

dummies1 = training2.drop('is_canceled', 1)
test_X = pd.get_dummies(dummies1)
test_y = training2['is_canceled']


,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,hotel_City Hotel,hotel_Resort Hotel,arrival_date_month_April,arrival_date_month_August,arrival_date_month_December,arrival_date_month_February,arrival_date_month_January,arrival_date_month_July,arrival_date_month_June,arrival_date_month_March,arrival_date_month_May,arrival_date_month_November,arrival_date_month_October,arrival_date_month_September,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,342,2015,27,1,0,0,2,0,0,0,0,3,0,0.0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,737,2015,27,1,0,0,2,0,0,0,0,4,0,0.0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,7,2015,27,1,0,1,1,0,0,0,0,0,0,75.0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,13,2015,27,1,0,1,1,0,0,0,0,0,0,75.0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,14,2015,27,1,0,2,2,0,0,0,0,0,0,98.0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107385,60,2017,10,10,0,2,2,0,0,0,0,0,0,97.2,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
107386,8,2017,10,8,0,4,1,0,0,0,0,0,0,83.7,0,2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
107387,77,2017,10,7,0,5,2,0,0,0,0,0,0,68.0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
107388,63,2017,10,7,0,5,2,0,0,0,0,0,0,68.0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


The logistic model was not effective, due to the fact that some months were not included in the 12000 reservations which we set aside.  This seems odd that some months were not included, but this could be due to when these reservations were recorded.


In [0]:
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import GridSearchCV 
import numpy as np

max_depth = list(np.arange(1,10))
max_depth.append(None)
param_grid = { 
    'max_depth': max_depth,
    'n_estimators': np.arange(100, 1100, 200), 
    'min_samples_split': [2,4,8,16],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier()


random_search = RandomizedSearchCV(rf, param_distributions = param_grid, 
                                   n_jobs = 1, verbose=50, n_iter=100, cv=3)


random_search.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] n_estimators=100, min_samples_split=16, min_samples_leaf=4, max_depth=4 
[CV]  n_estimators=100, min_samples_split=16, min_samples_leaf=4, max_depth=4, score=0.750, total=   5.5s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.5s remaining:    0.0s
[CV] n_estimators=100, min_samples_split=16, min_samples_leaf=4, max_depth=4 
[CV]  n_estimators=100, min_samples_split=16, min_samples_leaf=4, max_depth=4, score=0.753, total=   5.5s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   11.0s remaining:    0.0s
[CV] n_estimators=100, min_samples_split=16, min_samples_leaf=4, max_depth=4 
[CV]  n_estimators=100, min_samples_split=16, min_samples_leaf=4, max_depth=4, score=0.748, total=   5.4s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   16.4s remaining:    0.0s
[CV] n_estimators=100, min_samples_split=2, min_samples_leaf=2, 

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [0]:
best_random = random_search.best_estimator_ 
best_random

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=900,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
accuracy_score(y_test, best_random.fit(X_train, y_train).predict(X_test))

0.9075299438813971

In [6]:
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [0]:
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline

lbl_enc = preprocessing.LabelEncoder()
data['hotel'] = lbl_enc.fit_transform(data.hotel.values)
data['arrival_date_year'] = lbl_enc.fit_transform(data.arrival_date_year.values)
data['arrival_date_month'] = lbl_enc.fit_transform(data.arrival_date_month.values)
data['arrival_date_day_of_month'] = lbl_enc.fit_transform(data.arrival_date_day_of_month.values)
data['meal'] = lbl_enc.fit_transform(data.meal.values)
data['market_segment'] = lbl_enc.fit_transform(data.market_segment.values)
data['distribution_channel'] = lbl_enc.fit_transform(data.distribution_channel.values)
data['reserved_room_type'] = lbl_enc.fit_transform(data.reserved_room_type.values)
data['assigned_room_type'] = lbl_enc.fit_transform(data.assigned_room_type.values)
data['customer_type'] = lbl_enc.fit_transform(data.customer_type.values)
data['deposit_type'] = lbl_enc.fit_transform(data.deposit_type.values)

In [28]:
# Scaling numerical data to enhance machine learning algorithms
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
data[['lead_time', 'adr']] = min_max_scaler.fit_transform(data[['lead_time', 'adr']])
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,1,0,0.464043,0,5,27,0,0,0,2,0.0,0,0,PRT,3,1,0,0,0,2,2,3,0,NaN,NaN,0,2,0.001180,0,0,Check-Out,2015-07-01
1,1,0,1.000000,0,5,27,0,0,0,2,0.0,0,0,PRT,3,1,0,0,0,2,2,4,0,NaN,NaN,0,2,0.001180,0,0,Check-Out,2015-07-01
2,1,0,0.009498,0,5,27,0,0,1,1,0.0,0,0,GBR,3,1,0,0,0,0,2,0,0,NaN,NaN,0,2,0.015053,0,0,Check-Out,2015-07-02
3,1,0,0.017639,0,5,27,0,0,1,1,0.0,0,0,GBR,2,0,0,0,0,0,0,0,0,304.0,NaN,0,2,0.015053,0,0,Check-Out,2015-07-02
4,1,0,0.018996,0,5,27,0,0,2,2,0.0,0,0,GBR,6,3,0,0,0,0,0,0,0,240.0,NaN,0,2,0.019307,0,1,Check-Out,2015-07-03


In [0]:
train = data.drop(['agent', 'company', 'country', 'children', 'reservation_status', 'reservation_status_date'], axis = 1)
A = train.drop(['is_canceled'], axis = 1)
B = train['is_canceled']

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.1, random_state=2)

In [35]:
A_train

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,meal,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
48681,0,0.031208,1,7,13,25,2,2,2,0,0,6,3,0,0,0,0,0,0,0,0,2,0.026336,0,0
105530,0,0.000000,2,3,6,9,0,1,1,0,3,3,1,0,0,0,0,3,0,0,0,2,0.017457,0,0
28805,1,0.251018,1,10,41,5,2,3,2,0,0,6,3,0,0,0,0,5,0,0,0,2,0.011348,0,0
14876,1,0.179104,0,5,30,18,2,4,2,1,0,6,3,0,1,0,0,0,0,0,0,2,0.032994,0,1
37389,1,0.189959,2,6,24,11,1,3,1,0,0,6,3,0,0,0,0,0,1,0,0,3,0.019862,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44566,0,0.037992,0,10,41,6,0,3,1,0,0,6,3,0,0,0,0,3,0,0,0,0,0.021070,0,1
95816,0,0.006784,1,1,35,20,2,1,2,0,0,6,3,0,0,0,3,3,0,0,0,2,0.023256,0,1
72173,0,0.017639,2,5,30,22,1,0,2,0,3,6,3,0,0,0,0,0,0,0,0,2,0.027075,0,0
100879,0,0.360923,1,10,45,29,2,2,2,0,0,6,3,0,0,0,0,0,0,0,0,3,0.017217,0,2


In [36]:
forest = RandomForestClassifier(random_state=2)

forest.fit(A_train, B_train)
forest.score(A_test, B_test)

0.8720998408576933

In [37]:
best = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=900,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
best.fit(A_train, B_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=900,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [40]:
accuracy_score(B_test, best.fit(A_train, B_train).predict(A_test))

0.8715972862048748

In [53]:
importance2 = pd.DataFrame({'Importance': best.feature_importances_}, index=A_train.columns).sort_values('Importance', ascending = False)
importance2

,Importance
lead_time,0.154674
deposit_type,0.146320
adr,0.103031
arrival_date_day_of_month,0.071449
total_of_special_requests,0.063274
arrival_date_week_number,0.060854
market_segment,0.050913
stays_in_week_nights,0.044426
previous_cancellations,0.043625
arrival_date_month,0.031484


In [44]:
import plotly.express as px
important2 = importance2.head(20)

fig1 = px.bar(important2, x=important2.index, y = 'Importance')
fig1.show()

In [0]:
C = data[['lead_time', 'deposit_type', 'adr', 'arrival_date_day_of_month', 'total_of_special_requests']]
D = data['is_canceled']


In [0]:
C_train, C_test, D_train, D_test = train_test_split(C, D, test_size=0.1, random_state=2)

In [49]:
forest3 = RandomForestClassifier(random_state=2)

forest3.fit(C_train, D_train)
forest3.score(C_test, D_test)

0.8198341569645698

In [50]:
importance3 = pd.DataFrame({'Importance': forest3.feature_importances_}, index=C_train.columns).sort_values('Importance', ascending = False)
importance3

,Importance
adr,0.312003
lead_time,0.311076
deposit_type,0.189580
arrival_date_day_of_month,0.134805
total_of_special_requests,0.052537


In [51]:
important3 = importance3.head(20)

fig2 = px.bar(important3, x=important3.index, y = 'Importance')
fig2.show()